In [1]:
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from models import *
from datasets import *
from utils import *

import torch.nn as nn
import torch.nn.functional as F
import torch

from dataset.time_dataset import TsDataset
from pathlib import Path

import matplotlib.pyplot as plt

from dtw import dtw

In [2]:
# Dataset loader
datapath = Path('data')
dataset = TsDataset(datapath/'CaseI-Attacks without any change.csv')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                        shuffle=True, num_workers=4)
print("The size of the dataset is: ", dataset.data_normal.size(), dataset.data_abnormal.size())

The size of the dataset is:  torch.Size([33, 576]) torch.Size([33, 15, 576])


In [3]:
normal_dim = dataset.data_normal.size(1)
abnormal_dim = dataset.data_abnormal.size(2)
in_dim = dataset.__len__()

In [4]:
G_AB = LSTMGenerator(normal_dim, normal_dim)
G_BA = LSTMGenerator(abnormal_dim, abnormal_dim)
D_A = LSTMDiscriminator(normal_dim)
D_B = LSTMDiscriminator(abnormal_dim)

In [5]:
G_AB.load_state_dict(torch.load("saved_models/cyber/G_AB_190.pth"))
G_BA.load_state_dict(torch.load("saved_models/cyber/G_BA_190.pth"))
D_A.load_state_dict(torch.load("saved_models/cyber/D_A_190.pth"))
D_B.load_state_dict(torch.load("saved_models/cyber/D_B_190.pth"))

<All keys matched successfully>

In [6]:
G_AB.eval()
G_BA.eval()
D_A.eval()
D_B.eval()

LSTMDiscriminator(
  (lstm): LSTM(576, 256, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=256, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [7]:
dataset.data_normal.size()

torch.Size([33, 576])

In [8]:
threshold = 6.0

In [9]:
dataset_test = TsDataset(datapath/'CaseII.csv')
dataset_test.data_normal.size()

torch.Size([33, 144])

In [10]:
with torch.no_grad():
    output = G_AB(dataset_test.data_normal.repeat(1, 4).unsqueeze(1))
    output_back = G_BA(output)
print(f"Output size: {output.size()} \t Output Back size: {output_back.size()}")


for i in range(output.size(0)):
    plt.plot(output[i, 0, :].numpy())
    plt.plot(output_back[i, 0, :].numpy())
    plt.plot(dataset_test.data_normal[i, :].numpy())
    plt.legend(['Generated', 'Generated Back', 'Original'])
    plt.savefig(f'cyber_result/forward/output_{i}.png')
    plt.close()

    dist1, cost, acc, path = dtw(output[i,0,:].reshape(-1, 1), dataset_test.data_normal[i, :].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    dist2, cost, acc, path = dtw(output_back[i,0,:].reshape(-1, 1), output[i,0,:].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    dist3, cost, acc, path = dtw(output_back[i,0,:].reshape(-1, 1), dataset_test.data_normal[i, :].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    res = (dist2+dist3)/dist1
    print("+" if res < threshold else "-", res)

Output size: torch.Size([33, 1, 576]) 	 Output Back size: torch.Size([33, 1, 576])
+ 2.331029491432602
+ 1.2747943237464927
+ 1.3881818920110267
+ 1.5031051814492338
+ 1.1408047967187396
+ 1.1880372445278344
+ 1.1880372445278344
+ 1.1108820571387772
+ 1.1005703160214078
+ 1.1103148012190187
+ 1.093998404511999
+ 1.0872938626457205
+ 1.0631788787009715
+ 1.6354867703217508
+ 2.616469741893868
+ 3.02404477933798
+ 2.421994590137501
+ 2.256806239547291
+ 1.0388489358107535
+ 1.3002635822973956
+ 2.7207029640515823
+ 5.64093910070854
+ 2.1506033947235634
- 12.368059233448436
- 10.448845806203183
+ 5.404118493326567
+ 3.908509152939757
+ 2.7207029640515823
+ 5.761931616110566
- 12.225558915119224
+ 2.9776854156938946
+ 4.927758160269898
- 8.512895109023098


---

In [11]:
dataset_test.data_abnormal.size()

torch.Size([33, 15, 144])

In [12]:
data_abnormal_test = dataset.data_abnormal[:,4,:]
data_abnormal_test.size()

torch.Size([33, 576])

In [13]:
with torch.no_grad():
    output = G_AB(data_abnormal_test.unsqueeze(1))
    output_back = G_BA(output)

In [14]:
for i in range(output.size(0)):
    plt.plot(output[i, 0, :].numpy())
    plt.plot(output_back[i, 0, :].numpy())
    plt.plot(data_abnormal_test[i, :].numpy())
    plt.legend(['Generated', 'Generated Back', 'Original'])
    plt.savefig(f'cyber_result/detection/output_{i}.png')
    plt.close()

    dist1, cost, acc, path = dtw(output[i,0,:].reshape(-1, 1), data_abnormal_test[i, :].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    dist2, cost, acc, path = dtw(output_back[i,0,:].reshape(-1, 1), output[i,0,:].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    dist3, cost, acc, path = dtw(output_back[i,0,:].reshape(-1, 1), data_abnormal_test[i, :].reshape(-1, 1), dist=lambda x, y: np.linalg.norm(x - y, ord=1)) 
    res = (dist2+dist3)/dist1   
    print("+" if res < threshold else "-", res)

- 16.91317726903861
- 21.77773310600828
- 17.712344376989314
- 17.063933322802395
- 8.149383151311397
- 18.963418303263914
- 18.963418303263914
- 19.24529924156134
- 19.5690098475925
- 19.82742398379115
- 19.838169672339983
- 20.348981515432765
- 19.225526521673643
+ 1.7001143878785956
- 11.443426734977365
- 7.184549234687114
- 8.152891377692713
+ 4.630614909240406
- 9.922095570686894
+ 1.497892297466366
- 6.546857278488791
- 9.381993680709739
+ 1.744386673793939
+ 1.0721229564493846
+ 1.0067357629559972
+ 1.071640135561151
- 8.333047939356062
- 6.546857278488791
- 9.926762622135023
- 7.773616823570045
- 17.26726747278578
- 9.373393732369156
- 10.163532326486429
